In [225]:
import json
import fileinput
import requests
import re
import os

# helper function to convert our text files to json
def text2json(text):
    fin = fileinput.input(text, inplace=1)
    for line in fin:
        if fin.isfirstline():
            print ("[" + line)
        else:
            print ("," + line)
    fin.close()
    
    f = open(text,"a")
    f.write("]")
    f.close()
    
# load json from textfile
def loaddata(text): 
    # if json.load fails, format
    try:
        f = open(text, 'r')
        data = json.load(f)
        f.close()
        return data 
    except:
        print("Formatting " + text + "...")
        text2json(text)
        return loaddata(text)

# load title from url
def gettitle(url):
    r = requests.get(url)
    # return text between <title> tags in html 
    return re.findall("(?<=<title>)\s*(.*)\s*(?=<\/title>)", r.text)

# add url title to ['title'] attribute, textfile to new file
def addtitle(textfile, outdir, counter):
    data = loaddata(textfile)
    try:
        for tweet in data:
            titles = []
            for url in tweet['tweet'][0]['entities']['urls']:
                titles.append(gettitle(url['expanded_url']))
        #     print(titles)
            tweet['titles'] = titles

        outfile = outdir + "t-" + str(counter) + ".json"
        with open(outfile, 'w') as file:
             file.write(json.dumps(data)) # use `json.loads` to do the reverse
    except Exception as e:
        print("Exception thrown: " + str(e))
    

In [226]:
import os

# set up dir
print(os.getcwd())
directory = os.getcwd()
files = os.listdir(directory)

# for output writing
counter = 1
odir = "../convertedtweets/"

for f in files:
    filename, file_extension = os.path.splitext(f)
    if(file_extension == '.json'):
        addtitle(f, odir, counter)
        counter += 1
#     print(f)

/Users/lucassong/code/ir/twit-scraper/tweetsk
